In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data1.csv',lineterminator='\n')
df.head()

,id,dialect,txt
0,1175358310087892992,IQ,@Nw8ieJUwaCAAreT لكن بالنهاية .. ينتفض .. يغير .
1,1175416117793349632,IQ,@7zNqXP0yrODdRjK يعني هذا محسوب على البشر .. ح...
2,1175450108898565888,IQ,@KanaanRema مبين من كلامه خليجي
3,1175471073770573824,IQ,@HAIDER76128900 يسلملي مرورك وروحك الحلوه💐
4,1175496913145217024,IQ,@hmo2406 وين هل الغيبه اخ محمد 🌸🌺


# cleaning text

In [31]:
for letter in '#.][!XR':
    df['txt'] = df['txt'].astype(str).str.replace(letter,'')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


# Preprocess data


In [35]:
import re

#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)



In [36]:
import string
import re

arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text
def processPost(tweet): 

    #Replace @username with empty string
    tweet = re.sub('@[^\s]+', ' ', tweet)
    
    #Convert www.* or https?://* to " "
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',tweet)
    
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)

    # remove punctuations
    tweet= remove_punctuations(tweet)
    
    # normalize the tweet
    tweet= normalize_arabic(tweet)

    #remove emojies 
    tweet = emoji_pattern.sub(r'', tweet)
    
    return tweet

In [37]:
from nltk.tokenize import RegexpTokenizer

df["txt"] = df['txt'].apply(lambda x: processPost(x))

tokenizer = RegexpTokenizer(r'\w+')
df["txt"] = df["txt"].apply(tokenizer.tokenize)

In [38]:
df['txt'].head()

0                         [لكن, بالنهايه, ينتفض, يغير]
1    [يعني, هذا, محسوب, علي, البشر, حيونه, ووحشيه, ...
2                             [مبين, من, كلامه, خليجي]
3                       [يسلملي, مرورك, وروحك, الحلوه]
4                          [وين, هل, الغيبه, اخ, محمد]
Name: txt, dtype: object

In [39]:
import nltk 
# nltk.download('stopwords')
from nltk.corpus import stopwords

stopwords_list = stopwords.words('arabic')
# remove stop words
df["txt"]=df["txt"].apply(lambda x: [item for item in x if item not in stopwords_list])

In [40]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/data2.csv',index=False)